Imports

In [1]:
import numpy as NP
import ReducedEchelon as RE
import Nullspace as NS
import PrintDimensionalMatrix as PMat
from typing import OrderedDict
from scipy.linalg import null_space


Definition of variables

In [2]:
# define variables as a dictionary
# all variables are given in units of a dedicated basic unit system (usually SI)
# use whatever you feel makes sense, later on the code will determine, which ones you used.
# in case of doubt, check the output and see if all intended units are there or somewhere a
# type has occured.
variables = {}
# example Reynolds number of flow around sphere
# variables['mu']  = {'kg': 1, 's': -1, 'm': -1}
# variables['v']   = {'m': 1, 's': -1}
# variables['R']   = {'m': 1}
# variables['rho'] = {'kg': 1, 'm': -3}

# your variable system
variables['L'] = {'m': 1}                                   # representative macro scale length
variables['mu_l'] = {'kg': 1, 's': -1, 'm': -1}             # liquid viscosity
variables['cp_w'] = {'m': 2, 's': -2, 'K': -1}              # heat capacity of liquid
variables['D_eff'] = {'m': 2, 's': -1}                      # effective diffusion coefficient of species
variables['rho_l'] = {'kg': 1, 'm': -3}                     # liquid side density
variables['alpha'] = {'kg': 1, 's': -3, 'K': -1}            # external heat transfer coefficient
variables['beta']  = {'m': 1, 's': -1}                      # external mass transfer coefficient
variables['c_0']   = {'mol': 1, 'm': -3}                    # initial concentration
variables['c_sat'] = {'mol': 1, 'm': -3}                    # saturation concentration
variables['sigma'] = {'kg' : 1, 's' : -2}                   # surface tension
variables['a_v']   = {'m': -1}                              # specific surface area
variables['d_p']   = {'m': 1}                               # average pore diameter
variables['l_s']   = {'kg': 1, 'm': 1, 's': -3, 'K': -1}    # heat conductivity of solid
variables['l_w']   = {'kg': 1, 'm': 1, 's': -3, 'K': -1}    # heat conductivity of liquid
variables['cp_s']  = {'m': 2, 's': -2, 'K': -1}             # heat capacity of solid
variables['T_amb'] = {'K': 1}

# number of digits for printing the entries of the dimensional matrix
digits = 2

Assembling Matrix for Reduction

In [3]:
# converting to an ordered dictionary, so we can always rely on the order of entries
variables = OrderedDict(variables.items())

# giving some debugging feedback to the user
num_var = len(variables)
print('Following variables were provided')
for i, v in variables.items():
    print(f"{i} = {v}")

print("")

# determining the independet physical dimensions
independent_variables = []
for i, v in variables.items():
    for p, o in v.items():
        if not p in independent_variables:
            independent_variables.append(p)

num_var_independent = len(independent_variables)
print(f"{num_var_independent} independent dimensions were found: ", end = "")
for v in range(0, num_var_independent - 1):
    print(independent_variables[v], end = ", ")
print(independent_variables[-1])

print("")

# assembling the dimension matrix
A = NP.zeros((num_var_independent, num_var))
col = 0
for i, v in variables.items():
    for row in range(0, num_var_independent):
        A[row, col] = v.get(independent_variables[row], 0)
    col += 1

Ared, jb = RE.rref(A)


Following variables were provided
L = {'m': 1}
mu_l = {'kg': 1, 's': -1, 'm': -1}
cp_w = {'m': 2, 's': -2, 'K': -1}
D_eff = {'m': 2, 's': -1}
rho_l = {'kg': 1, 'm': -3}
alpha = {'kg': 1, 's': -3, 'K': -1}
beta = {'m': 1, 's': -1}
c_0 = {'mol': 1, 'm': -3}
c_sat = {'mol': 1, 'm': -3}
sigma = {'kg': 1, 's': -2}
a_v = {'m': -1}
d_p = {'m': 1}
l_s = {'kg': 1, 'm': 1, 's': -3, 'K': -1}
l_w = {'kg': 1, 'm': 1, 's': -3, 'K': -1}
cp_s = {'m': 2, 's': -2, 'K': -1}

5 independent dimensions were found: m, kg, s, K, mol



Result

In [4]:
print('Determined dimensional matrix is:')
PMat.Print(A, variables, independent_variables, digits)
print("")
print('The reduced row echelon form was found to be:')
PMat.Print(Ared, variables, independent_variables, digits)
print("")
num_pis = num_var - len(jb)
if num_pis == 1:
    print(f"The matrix is of rank {len(jb)} and provides 1 Pi number:")
else:
    print(f"The matrix is of rank {len(jb)} and provides {num_pis} Pi numbers:")

# The nullspace of the matrix then provides us with the dimensionless variables
# Be aware, this is a mere guess, the result is just one of infinite possibilities
pi_array = NS.nullspace(Ared, list(variables.items()))

if num_pis == 0:
    print('No dimensionless number can be defined for the provided system')
else:
    Pi = {}
    var_list = list(variables.keys())
    for i in range(0, num_pis):
        pi_local = {}
        normalizer = NP.max(pi_array)
        for j in range(0, len(var_list)):
            if not abs(pi_array[j, i]) < 1e-3:
                normalizer = min(abs(pi_array[j,i]), normalizer)
        for j in range(0, len(var_list)):
            if not abs(pi_array[j, i]) < 1e-3:
                pi_local[var_list[j]] = pi_array[j,i] / normalizer
        Pi["Pi %u"%(i+1)] = pi_local
    
    for i_p, p_p in Pi.items():
        print(f"{i_p} = ", end="")
        for i_pp, p_pp in p_p.items():
            print(f"{i_pp}^({round(p_pp, digits)})", end = " ")
        print("")

print("\n\nBe aware, the numbers provided by this code are merely one possibility! Especially in the case of multiple numbers, an infinite possibility of pi numbers can be forumulated!")


Determined dimensional matrix is:
	L	mu_l	cp_w	D_eff	rho_l	alpha	beta	c_0	c_sat	sigma	a_v	d_p	l_s	l_w	cp_s	
m	1.0	-1.0	2.0	2.0	-3.0	0.0	1.0	-3.0	-3.0	0.0	-1.0	1.0	1.0	1.0	2.0	
kg	0.0	1.0	0.0	0.0	1.0	1.0	0.0	0.0	0.0	1.0	0.0	0.0	1.0	1.0	0.0	
s	0.0	-1.0	-2.0	-1.0	0.0	-3.0	-1.0	0.0	0.0	-2.0	0.0	0.0	-3.0	-3.0	-2.0	
K	0.0	0.0	-1.0	0.0	0.0	-1.0	0.0	0.0	0.0	0.0	0.0	0.0	-1.0	-1.0	-1.0	
mol	0.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	


The reduced row echelon form was found to be:
	L	mu_l	cp_w	D_eff	rho_l	alpha	beta	c_0	c_sat	sigma	a_v	d_p	l_s	l_w	cp_s	
m	1.0	0.0	0.0	0.0	0.0	-1.0	-1.0	0.0	0.0	-1.0	-1.0	1.0	0.0	0.0	0.0	
kg	0.0	1.0	0.0	0.0	1.0	1.0	0.0	0.0	0.0	1.0	0.0	0.0	1.0	1.0	0.0	
s	0.0	0.0	1.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	1.0	1.0	
K	0.0	0.0	0.0	1.0	-1.0	0.0	1.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	
mol	0.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	


The matrix is of rank 5 and provides 10 Pi numbers:
Pi 1 = mu_l^(-1.0) D_eff^(1.0) rho_l^(1.0) 
Pi 2 = L^(1.

Test Pi numbers for nullity criteria

In [5]:
# Enter Pi-numbers
Pi = []
Pi.append({'c_0' : 1, 'c_sat' : -1 })
Pi.append({'a_v' : 1, 'T_amb' : -1, 'mu_l' : 1, 'beta' : 2, 'alpha' : -1})
Pi.append({'d_p': -1, 'T_amb': -1, 'mu_l': 1, 'beta': 2, 'alpha': -1})
Pi.append({'L': 1, 'T_amb': 1, 'mu_l': -1, 'beta': -2, 'alpha': 1})
Pi.append({'rho_l': 1, 'T_amb': -1, 'beta': 3, 'alpha': -1})
Pi.append({'l_s': 1, 'T_amb': 1, 'mu_l': -1, 'beta': -2})
Pi.append({'l_w': 1, 'T_amb': 1, 'mu_l': -1, 'beta': -2})
Pi.append({'cp_s': 1, 'T_amb': 1, 'beta': -2})
Pi.append({'cp_w': 1, 'T_amb': 1, 'beta': -2})
Pi.append({'D_eff' : 1, 'T_amb' : 1, 'mu_l' : -1, 'alpha' : 1, 'beta' : -3})


In [6]:
# Evaluation numbers

# test if number is sufficient
if not len(Pi) == num_pis:
    print(f"Inconsistent amount of Pi-numbers provided, expected {num_pis}, found {len(Pi)}")

# test if all components are known
var_list = list(variables.keys())
for e in Pi:
    for i, p in e.items():
        if not i in var_list:
            print(f"Cannot find {i} in the predefined variables")

# assemble null-space
nullspace = NP.zeros((num_var, num_pis))
for i in range(len(Pi)):
    for key, v in Pi[i].items():
        nullspace[var_list.index(key), i] = v

# test nullspace
test_space = A.dot(nullspace)
if NP.any(abs(test_space) > 1e-10):
    print(f"Set of Pi numbers does not form a nullspace, see result for reference:")
    print(test_space)
else:
    print("All Pi numbers form a null-space\n")



Cannot find T_amb in the predefined variables
Cannot find T_amb in the predefined variables
Cannot find T_amb in the predefined variables
Cannot find T_amb in the predefined variables
Cannot find T_amb in the predefined variables
Cannot find T_amb in the predefined variables
Cannot find T_amb in the predefined variables
Cannot find T_amb in the predefined variables
Cannot find T_amb in the predefined variables


ValueError: 'T_amb' is not in list